In [ ]:
%matplotlib inline
%pylab inline

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.figsize'] = (7, 3)   # Change the size of plots
matplotlib.rcParams['figure.dpi'] = 108

In [ ]:
import numpy as np
import numpy.matlib

In [ ]:
# Time interval between readings
d = 1.0 / 5000.0
# Stride size of the running window
M = 2000
N = 2000
# Time-series data usually come in chunks
chunk_size = 16

# A scaling constant of the clean reference
tics_per_second = 1000000

# Have the time-series to be at least X seconds
K = int(floor(max(5 * N, round(1.0 / d)) / chunk_size / 2) * chunk_size * 2)

# Emulate staggered PRT: 2, 3, 2, 3, ...
y = np.cumsum(np.kron(np.ones(int(K / 2), ), [2, 3])) * (0.5 * d)

# Clean reference of tic count
u = (y + 19760520) * tics_per_second

# Time in double with an arbitrary offset
y = y + 7000.0

# Add noise to x, on the orders of milliseconds
# x = y + 10.0e-3 * (np.random.random(y.shape) - 0.5)
L = round(K / chunk_size, )
jitter = np.kron(0.5e-3 * (np.random.random(L, ) - 0.5), np.ones(chunk_size, ))
burst = np.kron(np.ones(L, ), np.linspace(0, 0.05e-3, chunk_size))

x = y + jitter + burst

In [ ]:
N = 256
matplotlib.pyplot.plot(noise[:N], label='Jitter')
matplotlib.pyplot.plot(burst[:N], label='Burst')
matplotlib.pyplot.xlim([0, N])
matplotlib.pyplot.legend()
matplotlib.pyplot.grid()

In [ ]:
h_x0 = np.ones(x.shape) * np.nan
h_u0 = np.ones(x.shape) * np.nan
h_dx = np.ones(x.shape) * np.nan
t = np.ones(x.shape) * np.nan
b = 1.0 / M
a = 1.0 - b

# Block size
L = 16

for i in range(2, len(x)):
    if i < N:
        dx_du = (x[i] - x[0]) / (u[i] - u[0])
    else:
        dx_du = (x[i] - x[i - N]) / (u[i] - u[i - N])
    if i <= L:
        x0 = x[i]
        u0 = (u[i] - u[0]) / i + u[0]
        dx = a * (x[i - 1] - x[0]) / (u[i - 1] - u[0]) + b * dx_du
    else:  
        x0 = a * x0 + b * x[i]
        u0 = a * u0 + b * u[i]
        dx = a * dx + b * dx_du
    t[i] = x0 + dx * (u[i] - u0)
    # Keep a history
    h_dx[i] = dx
    h_x0[i] = x0
    h_u0[i] = u0
    
# For plotting, set the beginning t values to be nice
t[1] = t[2] - d
t[0] = t[1] - d

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(11, 6))

xx = y - y[0]

ax1 = fig.add_subplot(221)
h1 = ax1.plot(xx, 1.0e3 * (t - y), '-m')
ax1.grid()
ax1.text(y[-1] - y[0], 1.3, 'M = {}, N = {}'.format(M, N), ha='right')
matplotlib.pyplot.ylabel('Time Error (ms)')
matplotlib.pyplot.ylim([-3.0, 3.0])
ax1b = ax1.twinx()
# h2 = ax1b.plot(xx, (u - h_u0) * 1.0e-6 / M, 'g')
h2 = ax1b.plot(xx, (u - h_u0) * 1.0e-6 * d * M, 'g')
# matplotlib.pyplot.ylim([0, 1.0e9])
ax1b.legend(h1 + h2, ['Time', 'Reference'], loc=4)
matplotlib.pyplot.title('Accuracy of Time & Reference')

ax2 = fig.add_subplot(222)
ax2.plot(xx, h_dx, label='Estimate')
ax2.plot([xx[0], xx[-1]], np.array([1.0, 1.0]), '--', label='True dx / du')
ax2.grid()
ax2.legend()
# matplotlib.pyplot.ylim(np.array([0.99, 1.01]))
matplotlib.pyplot.title('Time History of dx / du')

ax3 = fig.add_subplot(223)
h31 = ax3.plot(y - y[0], (h_u0 + tics_per_second * M))
h32 = ax3.plot([0, K * d], np.array([u[0], u[-1]]), '--')
ax3.grid()
ax3.legend(h31 + h32, ['Estimate', 'True u0'])

ax4 = fig.add_subplot(224)
h41 = ax4.plot(y - y[0], h_x0 - 7000.0 + d * M)
h42 = ax4.plot([0, K * d], [0, K * d], '--')
ax4.grid()
ax4.legend(h41 + h42, ['Estimate', 'True x0'])

In [ ]:
fig.savefig('/Users/boonleng/Desktop/M{0:02.0f}-N{1:02.0f}.png'.format(1.0e-3 * M, 1.0e-3 * N))

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(11, 3))
h1 = matplotlib.pyplot.plot(y[1:] - y[0], 1.0e3 * np.diff(y), '-o', label='Noisy Time')
h2 = matplotlib.pyplot.plot(y[2:] - y[0], 1.0e3 * np.diff(t)[1:], '-o', label='Pred. Time')
matplotlib.pyplot.grid()
matplotlib.pyplot.legend()
matplotlib.pyplot.title('Time History of Noisy Arrival Time and Predicted Time')
matplotlib.pyplot.xlim(np.array([0.001, 0.01]) + 1)
matplotlib.pyplot.ylim(np.array([-0.1, 1]))

print('Original jitter =', np.std(np.diff(x[-N:])) * 1.0e3, ' ms')
print('Smoothed jitter =', np.std(np.diff(t[-N:])) * 1.0e3, ' ms')

In [ ]:
np.diff(u[:10]) * 1e-6